In [1]:
import scipy.cluster.hierarchy as sch
import numpy as np
import pandas as pd
from datetime import date
from matplotlib import pyplot as plt
import cvxopt as opt # optimization library
from cvxopt import blas, solvers
from alpha_vantage.timeseries import TimeSeries # free stock data api
import ffn # financial functions
# import config

/usr/local/lib/python3.7/site-packages/ffn/core.py:27: MatplotlibDeprecationWarning: The 'warn' parameter of use() is deprecated since Matplotlib 3.1 and will be removed in 3.3.  If any parameter follows 'warn', they should be pass as keyword, not positionally.
  matplotlib.use('agg', warn=False)
/usr/local/lib/python3.7/site-packages/pandas_datareader/compat/__init__.py:7: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  from pandas.util.testing import assert_frame_equal


## Define Hierarchical Risk Parity functions

In [2]:
# compute the inverse-variance portfolio
def getIVP(cov, **kargs):
    ivp = 1. / np.diag(cov)
    ivp /= ivp.sum()
    return ivp

In [3]:
# compute variance per cluster
def getClusterVar(cov,cItems):
    cov_ = cov.loc[cItems,cItems] # matrix slice
    w_ = getIVP(cov_).reshape(-1,1)
    cVar = np.dot(np.dot(w_.T,cov_),w_)[0,0]
    return cVar

In [4]:
# sort clusters by distance
def getQuasiDiag(link):
    link = link.astype(int)
    sortIx = pd.Series([link[-1, 0], link[-1, 1]])
    numItems = link[-1, 3]  # number of original items
    while sortIx.max() >= numItems:
        sortIx.index = range(0, sortIx.shape[0] * 2, 2)  # make space
        df0 = sortIx[sortIx >= numItems]  # find clusters
        i = df0.index
        j = df0.values - numItems
        sortIx[i] = link[j, 0]  # item 1
        df0 = pd.Series(link[j, 1], index=i + 1)
        sortIx = sortIx.append(df0)  # item 2
        sortIx = sortIx.sort_index()  # re-sort
        sortIx.index = range(sortIx.shape[0])  # re-index
    return sortIx.tolist()

In [5]:
# compute HRP allocation
def getRecBipart(cov, sortIx):
    w = pd.Series(1, index=sortIx)
    cItems = [sortIx]  # initialize all items in one cluster
    while len(cItems) > 0:
        cItems = [i[j:k] for i in cItems for j, k in ((0, len(i) // 2), (len(i) // 2, len(i))) if len(i) > 1]  # bi-section
        for i in range(0, len(cItems), 2):  # parse in pairs
            cItems0 = cItems[i]  # cluster 1
            cItems1 = cItems[i + 1]  # cluster 2
            cVar0 = getClusterVar(cov, cItems0)
            cVar1 = getClusterVar(cov, cItems1)
            alpha = 1 - cVar0 / (cVar0 + cVar1)
            w[cItems0] *= alpha  # weight 1
            w[cItems1] *= 1 - alpha  # weight 2
    return w

In [6]:
# define distance measure based on correlation, where 0<=d[i,j]<=1
def correlDist(corr):
    dist = ((1 - corr) / 2.)**.5  # distance matrix
    return dist

In [7]:
# construct hierarchical portfolio
def getHRP(cov, corr):
    dist = correlDist(corr)
    link = sch.linkage(dist, 'single')
    #dn = sch.dendrogram(link, labels=cov.index.values, label_rotation=90)
    #plt.show()
    sortIx = getQuasiDiag(link)
    sortIx = corr.index[sortIx].tolist()
    hrp = getRecBipart(cov, sortIx)
    return hrp.sort_index()

## Define Markowitz Portfolio functions, for comparison

In [8]:
def getMVP(cov):

    cov = cov.T.values
    n = len(cov)
    N = 100
    mus = [10 ** (5.0 * t / N - 1.0) for t in range(N)]

    # convert to cvxopt matrices
    S = opt.matrix(cov)
    pbar = opt.matrix(np.ones(cov.shape[0]))

    # create constraint matrices
    G = -opt.matrix(np.eye(n))  # negative n x n identity matrix
    h = opt.matrix(0.0, (n, 1))
    A = opt.matrix(1.0, (1, n))
    b = opt.matrix(1.0)

    # use quadratic programming to calculate efficient frontier weights
    portfolios = [solvers.qp(mu * S, -pbar, G, h, A, b)['x'] for mu in mus]
    # calculate risks and returns for the frontier
    returns = [blas.dot(pbar, x) for x in portfolios] # note that we need a returns "forecast"!
    risks = [np.sqrt(blas.dot(x, S * x)) for x in portfolios]
    # calculate 2nd degree polynomial of the frontier curve
    m1 = np.polyfit(returns, risks, 2)
    x1 = np.sqrt(m1[2] / m1[0])
    # Ccalculate the optimal portfolio
    wt = solvers.qp(opt.matrix(x1 * S), -pbar, G, h, A, b)['x']

    return list(wt)

## Compare approaches

In [9]:
# first, define a function to compare across all portfolio construction approaches
def get_all_portfolios(returns):
    
    cov, corr = returns.cov(), returns.corr()
    hrp = getHRP(cov, corr)
    ivp = getIVP(cov)
    ivp = pd.Series(ivp, index=cov.index)
    mvp = getMVP(cov)
    mvp = pd.Series(mvp, index=cov.index)
    
    portfolios = pd.DataFrame([mvp, ivp, hrp], index=['MVP', 'IVP', 'HRP']).T
    
    return portfolios

In [10]:
# define stock universe
stocks = {
    "Apple": "AAPL",
    "Amazon": "AMZN",
    "Alphabet": "GOOG",
    "Microsoft": "MSFT",
    "Facebook": "FB",
    #"Alibaba": "BABA",
    #"Berkshire Hathaway": "BRK-A",
    #"Tencent": "TCEHY",
    #"JPMorgan": "JPM",
    #"ExxonMobil": "XOM",
    #"Johnson & Johnson": "JNJ",
    #"Samsung Electronics": "005930.KS",
    #"Bank of America": "BAC"
}

In [11]:
# get stock data
stocks = pd.DataFrame(list(stocks.items()), columns=['name', 'symbol'])
ts = TimeSeries(key = '######', output_format = 'pandas') # note this is where need to put Alpha Vantage API key
stocks_close = pd.DataFrame()

for symbol in stocks.symbol.values:
    data, _ = ts.get_daily(symbol=symbol, outputsize='full')
    close = data['4. close']
    close.index = pd.to_datetime(close.index)
    stocks_close = stocks_close.append(close)

stocks_close = stocks_close.T
stocks_close = stocks_close.sort_index()
stocks_close = stocks_close.fillna(method='ffill')
stocks_close.columns = stocks.name.values
stocks_close = stocks_close["2015-01-01":"2020-03-31"]
returns = stocks_close.to_returns().dropna()

In [12]:
returns.head(5)

,Apple,Amazon,Alphabet,Microsoft,Facebook
2015-01-05,-0.028172,-0.020517,-0.020846,-0.009303,-0.016061
2015-01-06,0.000094,-0.022833,-0.023177,-0.014571,-0.013473
2015-01-07,0.014022,0.010600,-0.001713,0.012705,0.000000
2015-01-08,0.038422,0.006836,0.003153,0.029418,0.026592
2015-01-09,0.001072,-0.011749,-0.012951,-0.008405,-0.005564


In [13]:
# compute portfolios
portfolios = get_all_portfolios(returns)

/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:4: ClusterWarning: scipy.cluster: The symmetric non-negative hollow observation matrix looks suspiciously like an uncondensed distance matrix
  after removing the cwd from sys.path.


     pcost       dcost       gap    pres   dres
 0: -9.9999e-01 -2.0000e+00  1e+00  1e-16  1e+00
 1: -9.9999e-01 -1.0100e+00  1e-02  6e-17  1e-02
 2: -9.9999e-01 -1.0001e+00  1e-04  7e-17  1e-04
 3: -9.9999e-01 -9.9999e-01  2e-06  7e-17  2e-06
 4: -9.9999e-01 -9.9999e-01  2e-07  1e-16  1e-07
 5: -9.9999e-01 -9.9999e-01  6e-09  9e-17  9e-10
Optimal solution found.
     pcost       dcost       gap    pres   dres
 0: -9.9999e-01 -2.0000e+00  1e+00  1e-15  1e+00
 1: -9.9999e-01 -1.0100e+00  1e-02  6e-17  1e-02
 2: -9.9999e-01 -1.0001e+00  1e-04  1e-16  1e-04
 3: -9.9999e-01 -9.9999e-01  2e-06  9e-17  3e-06
 4: -9.9999e-01 -9.9999e-01  2e-07  1e-16  1e-07
 5: -9.9999e-01 -9.9999e-01  6e-09  1e-16  1e-09
Optimal solution found.
     pcost       dcost       gap    pres   dres
 0: -9.9999e-01 -2.0000e+00  1e+00  9e-16  1e+00
 1: -9.9999e-01 -1.0100e+00  1e-02  2e-16  1e-02
 2: -9.9999e-01 -1.0001e+00  1e-04  7e-17  1e-04
 3: -9.9999e-01 -9.9999e-01  2e-06  6e-17  3e-06
 4: -9.9999e-01 -9.9999e

 1: -7.5199e-01 -7.7727e-01  3e-02  5e-17  3e-02
 2: -7.5393e-01 -7.5557e-01  2e-03  1e-16  5e-17
 3: -7.5397e-01 -7.5399e-01  3e-05  2e-16  7e-17
 4: -7.5397e-01 -7.5397e-01  3e-07  1e-16  1e-16
Optimal solution found.
     pcost       dcost       gap    pres   dres
 0: -7.2137e-01 -1.7397e+00  1e+00  0e+00  1e+00
 1: -7.2188e-01 -7.4845e-01  3e-02  2e-16  3e-02
 2: -7.2391e-01 -7.2562e-01  2e-03  4e-17  7e-17
 3: -7.2395e-01 -7.2397e-01  3e-05  8e-17  3e-17
 4: -7.2395e-01 -7.2395e-01  3e-07  9e-17  7e-17
Optimal solution found.
     pcost       dcost       gap    pres   dres
 0: -6.8753e-01 -1.7075e+00  1e+00  0e+00  1e+00
 1: -6.8812e-01 -7.1604e-01  3e-02  6e-17  3e-02
 2: -6.9023e-01 -6.9199e-01  2e-03  8e-17  3e-17
 3: -6.9026e-01 -6.9029e-01  3e-05  2e-16  8e-17
 4: -6.9026e-01 -6.9026e-01  3e-07  8e-17  6e-17
Optimal solution found.
     pcost       dcost       gap    pres   dres
 0: -6.4958e-01 -1.6714e+00  1e+00  0e+00  1e+00
 1: -6.5026e-01 -6.7957e-01  3e-02  4e-17  4e-02


/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:8: RankWarning: Polyfit may be poorly conditioned
  


In [14]:
portfolios

,MVP,IVP,HRP
Apple,0.260790,0.202113,0.243984
Amazon,0.134563,0.175866,0.222179
Alphabet,0.311311,0.233598,0.164062
Microsoft,0.171536,0.215568,0.151398
Facebook,0.121800,0.172856,0.218377
